# Login / Switch DEV_TEST_PROD environment (1-timer) - ENSURE we have access to DEV

In [5]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLProject

p = ESMLProject()
p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
p.ws = ws

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL


# 1) ESML - TRAIN Classification, TITANIC model, and DEPLOY with predict_proba scoring

In [6]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.active_model = 10
p.inference_mode = False
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
Environment: dev
Inference version: 1

 - ds01_titanic
projects/project002/10_titanic_model_clas/train/ds01_titanic/in/dev/2021/01/01/
projects/project002/10_titanic_model_clas/train/ds01_titanic/out/bronze/dev/
projects/project002/10_titanic_model_clas/train/ds01_titanic/out/silver/dev/

 - ds02_haircolor
projects/project002/10_titanic_model_clas/train/ds02_haircolor/in/dev/2021/01/01/
projects/project002/10_titanic_model_clas/train/ds02_haircolor/out/bronze/dev/
projects/project002/10_titanic_model_clas/train/ds02_haircolor/out/silver/dev/

 - ds03_housing
projects/project002/10_titanic_model_clas/train/ds03_housing/in/dev/2021/01/01/
projects/project002/10_titanic_model_clas/train/ds03_housing/out/bronze/dev/
projects/project002/10_titanic_model_clas/train/ds03_housing/out/silver/dev/

 - ds04_lightsaber
projects/project002/10_titanic_model_clas/train/ds04_lightsaber/in/dev/2021/01/01/
projects/pro

## 1) ESML Scoring Drift/Concept Drift: Compare with `1-codeline`: Promote model or not? If better, then `Register model`
- `IF` newly trained model in `current` environment (`DEV`, `TEST` or `PROD`) scores BETTER than existing model in `target` environment, then `new model` can be registered and promoted.
- Q: Do we have `SCORING DRIFT / CONCEPT DRIFT?`
- Q: Is a model trained on NEW data better? IS the one in production degraded? (not fit for the data it scores - real world changed, other CONCEPT)
- A: - Lets check. Instead of `DataDrift`, lets look at `actual SCORING` on new data (and/or new code, feature engineering) - See if we should PROMOTE newly trained model...

In [7]:
print("current AI Factory environment: '{}' - AML WS: '{}'".format(p.dev_test_prod, p.ws.name))

current AI Factory environment: 'dev' - AML WS: 'msft-weu-DEV-eap-proj02_ai-amls'


In [8]:
from baselayer_azure_ml_model import ESMLModelCompare

current_env = p.dev_test_prod # dev
target_env = "test" # Does newly trained Model v3 in DEV, score better than Model v2 in TEST?

compare = ESMLModelCompare(p)
promote,source_model_name,new_run_id,target_model_name, target_best_run_id,target_workspace,source_model = compare.compare_scoring_current_vs_new_model(target_env) # Compare DEV to TEST (or TEST to PROD) - # 46 seconds (DEV:140 models, TEST: 20 models)

print("SCORING DRIFT: If new model scores better in DEV (new data, or new code), we can promote this to TEST & PROD \n")
print("New Model: {} in environment {}".format(target_model_name, p.dev_test_prod))
print("Existing Model: {} in environment {}".format(source_model_name,target_env))

if (promote): # Can register="promote" a model in same workspace (test->test), or also register in OTHER Azure ML workspace (test->prod)
    compare.register_model_in_correct_ws("test")  # 46 seconds

Compare model version in DEV with latest registered in TEST subscription/workspace

New trained model & cached RUN, has TASK_TYPE: classification and Best_Run_id: AutoML_4b3c03df-eaa6-43fd-b83b-a4857db588cc_18
Target model & RUN, in Azure ML Studio workspace to compare with, has TASK_TYPE: classification and Best_Run_id:AutoML_4b3c03df-eaa6-43fd-b83b-a4857db588cc_18 

Q: Do we have SCORING DRIFT / CONCEPT DRIFT? - Is a model trained on NEW data better? = the one in production degraded?

New trained model: 
INFO: Using ESML TEST_SET SCORING, since tagged on MODEL - using this to compare SCORING
AUC (AUC_weighted): 0.860456
Accuracy: 0.831461
Precision (precision_score_weighted): 0.716334
Recall (recall): 0.736111
F1 Score (1.0 is good): 0.779412
Logg loss (0.0 is good): -1
matthews_correlation (1.0 is good): 0.646734

Target model, to compare with; 
INFO: Using ESML TEST_SET SCORING, since tagged on MODEL - using this to compare SCORING
AUC (AUC_weighted): 0.860456
Accuracy: 0.831461
Pr

# 1) REGISTER MODEL trained in `DEV to  TEST` (usually TEST -> PROD)

In [9]:
p.dev_test_prod

'dev'

In [ ]:
from baselayer_azure_ml_model import ESMLModelCompare
compare = ESMLModelCompare(p) # TEST to compare with PROD
compare.register_model_in_correct_ws("test")

# `2) BATCH SCORING:` DEPLOY model trained in DEV workspace, as a AzureML Batch scoring pipeline `in TEST workspace`
- Swithch to TEST workspace, and deploy the `MOVED´ model (DEV to TEST)..that was registered in TEST workspace

In [12]:
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p.dev_test_prod = "test" # Swith to TEST environment
p.inference_mode = True
p_factory = ESMLPipelineFactory(p) # Will now find the correct Azure ML Workspace TEST to work with

##### PARAMETERS - Simulate dynamic parameters passed from Azure Datafactoy=Caller....such as "daily scoring" with a specific model_version
p_factory.batch_pipeline_parameters[0].default_value = 1 # 0 meaning latest model_version
scoring_date = '2021-01-01 10:35:01.243860'
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
##### PARAMETERS end

## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False)
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING) # Creates IN_2_GOLD_SCORING pipeline from template


Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = FALSE. [active_in_folder.json,active_scoring_in_folder.json] not found in LAKE. 
 - Using [active_in_folder.json,active_scoring_in_folder.json] from ArgParse or GIT. No override from datalake settings
Path for active folder (where no files exists):
AdlsGen2-ListFiles (req=1, existingItems=0)' for 'https://msftweudevcmnai2.dfs.core.windows.net/lake3?directory=projects/project002/10_titanic_model_clas/train/active&recursive=true&resource=filesystem' on storage failed with status code 'NotFound' (The specified path does not exist.), client request ID '685354e9-ea6f-481f-bc7e-be76a8c05e07', request ID '46c3446e-101f-002b-4502-ede01c000000
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVE

In [13]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

Created step IN 2 SILVER - ds01_titanic [20597ff1][1579864d-22cf-400f-b788-3a3663b12785], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_haircolor [6d7f4910][ccf85205-00ca-4c7c-93ea-b3765a870bd3], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds03_housing [b0671d04][d0d5df03-9d81-4947-b8de-253887d002f2], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds04_lightsaber [9e737517][9aabf636-bb13-4616-9735-cff872989749], (This step will run and generate new outputs)
Created step SILVER MERGED 2 GOLD [976d31c6][e4337f8e-41f3-43b0-a718-305a6fc5d0f8], (This step will run and generate new outputs)
Created step SCORING GOLD [3268c3f7][76a31748-5f9f-4b0d-8101-e88c7e7cb1d4], (This step will run and generate new outputs)
Submitted PipelineRun e1d3672d-9ec9-47eb-9db5-6988a6ba6744
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e1d3672d-9ec9-47eb-9db5-6988a6ba6744?wsid=/subscriptions/ca0a8c40-b06a-4e4e

'Finished'